In [2]:
#Basic packages
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, KFold

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA, NMF
from sklearn.manifold import TSNE
from umap import UMAP
from scipy.cluster.hierarchy import dendrogram, ward


import optuna
from optuna.integration import CatBoostPruningCallback

import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

sns.set_theme(style = 'white', palette = 'viridis')
pal = sns.color_palette('viridis')

pd.set_option('display.max_rows', 100)

In [3]:
#Models

import xgboost as xgb
import lightgbm as lgb
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor, LassoCV
from sklearn.linear_model import PassiveAggressiveRegressor, ARDRegression, RidgeCV, ElasticNetCV
from sklearn.linear_model import TheilSenRegressor, RANSACRegressor, HuberRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor, VotingRegressor, StackingRegressor

from sklearn.cross_decomposition import PLSRegression
from catboost import CatBoost, CatBoostRegressor, CatBoostClassifier
from catboost import Pool

#Metrics

from sklearn.metrics import mean_absolute_error

In [4]:
# File path and files

#File Path
data_dir = "/kaggle/input/playground-series-s3e14"
train_file = "train.csv"
test_file = "test.csv"
orig_file = "/kaggle/input/wild-blueberry-yield-prediction-dataset/WildBlueberryPollinationSimulationData.csv"
submission_file = "sample_submission.csv"

In [27]:
# test_data = Path(data_dir)/test_file
# train_data = Path(data_dir)/train_file
# submission_data = Path(data_dir)/submission_file

In [28]:
# train = pd.read_csv(train_data)
# test = pd.read_csv(test_data)
# submission_df = pd.read_csv(submission_data)

In [5]:
def get_data(data_dir = data_dir,train_file = train_file,  test_file = test_file, submission_file = submission_file):
    test_data = Path(data_dir)/test_file
    train_data = Path(data_dir)/train_file
    orig_data = Path(orig_file)
    submission_data = Path(data_dir)/submission_file
    train = pd.read_csv(train_data, index_col = "id")
    test = pd.read_csv(test_data, index_col = "id")
    orig_train = pd.read_csv(orig_data)
    submission_df = pd.read_csv(submission_data)
    return train,test,submission_df,orig_train
    

In [6]:
train, test, submission_df, orig_train = get_data()

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15289 entries, 0 to 15288
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   clonesize             15289 non-null  float64
 1   honeybee              15289 non-null  float64
 2   bumbles               15289 non-null  float64
 3   andrena               15289 non-null  float64
 4   osmia                 15289 non-null  float64
 5   MaxOfUpperTRange      15289 non-null  float64
 6   MinOfUpperTRange      15289 non-null  float64
 7   AverageOfUpperTRange  15289 non-null  float64
 8   MaxOfLowerTRange      15289 non-null  float64
 9   MinOfLowerTRange      15289 non-null  float64
 10  AverageOfLowerTRange  15289 non-null  float64
 11  RainingDays           15289 non-null  float64
 12  AverageRainingDays    15289 non-null  float64
 13  fruitset              15289 non-null  float64
 14  fruitmass             15289 non-null  float64
 15  seeds              

In [8]:
def create_EDA_summary (df = None):
    train_import_table = pd.DataFrame()
    train_import_table["dtype"] = df.apply(lambda x: x.dtype)
    train_import_table["NROW"] = df.shape[0]
    train_import_table["Unique_values"] = [ len(df[col].unique()) for col in df.columns]
    train_import_table["Percent_missing"] = (df.isnull().sum()/df.shape[0])*100
    
    return train_import_table

In [9]:
create_EDA_summary(df = train)

,dtype,NROW,Unique_values,Percent_missing
clonesize,float64,15289,6,0.0
honeybee,float64,15289,7,0.0
bumbles,float64,15289,11,0.0
andrena,float64,15289,16,0.0
osmia,float64,15289,14,0.0
MaxOfUpperTRange,float64,15289,6,0.0
MinOfUpperTRange,float64,15289,5,0.0
AverageOfUpperTRange,float64,15289,5,0.0
MaxOfLowerTRange,float64,15289,6,0.0
MinOfLowerTRange,float64,15289,7,0.0


In [10]:
create_EDA_summary(df = orig_train)

,dtype,NROW,Unique_values,Percent_missing
Row#,int64,777,777,0.0
clonesize,float64,777,6,0.0
honeybee,float64,777,7,0.0
bumbles,float64,777,10,0.0
andrena,float64,777,12,0.0
osmia,float64,777,12,0.0
MaxOfUpperTRange,float64,777,5,0.0
MinOfUpperTRange,float64,777,5,0.0
AverageOfUpperTRange,float64,777,5,0.0
MaxOfLowerTRange,float64,777,5,0.0


In [35]:
#cat_columns = test.iloc[:,:-3].columns.to_list()
#num_colums = test.iloc[:,-3:].columns.to_list()

In [11]:
orig_train = orig_train.drop(["Row#"], axis = 1)


In [12]:
# Define the target variable
target = "yield"
# Concatenate train and oiginal dataset
train = pd.concat([train, orig_train]).reset_index(drop = True)

In [13]:
#Feature engineering steps - All data

for df in [train, test]:
    df["fruit_seed"] = df["fruitset"] * df["seeds"]

In [14]:
#Separate the training data set into the features and target
train_X = train.drop([target], axis = 1).reset_index(drop=True)
train_y = train[target].reset_index(drop=True)

In [15]:
#Lists for feature engineering
num_list = test.columns.to_list()

In [16]:
#Feature engineering steps for cross-validation
numeric_transformer = Pipeline([('scaler', StandardScaler())])

#Preprocessing pipeline
preprocessor = ColumnTransformer([('num', numeric_transformer, num_list)])


In [17]:
#Define the cross-validation strategy
#Cross validation strategy
seed = 42
splits = 5
#cv = RepeatedStratifiedKFold(n_splits = splits, n_repeats = 5, random_state = seed)
cv = KFold(n_splits = splits, random_state = seed, shuffle = True)



In [18]:
#Define Hyperparamaters for Optuna

# xgb_params = {
#         #'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
#         'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
#         'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
#         'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
#         'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
#         'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
#         'n_estimators': 10000,
#         'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
#         'random_state': trial.suggest_categorical('random_state', [2020]),
#         'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
#     }

# cat_params  {
#         'loss_function': 'RMSE',
#         #'task_type': 'GPU',
#         'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
#         'max_bin': trial.suggest_int('max_bin', 200, 400),
#         #'rsm': trial.suggest_uniform('rsm', 0.3, 1.0),
#         'subsample': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
#         'learning_rate': trial.suggest_uniform('learning_rate', 0.006, 0.018),
#         'n_estimators':  25000,
#         'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15]),
#         'random_state': trial.suggest_categorical('random_state', [2020]),
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 300),
#     }


# lgb_params = {
#         'metric': 'rmse', 
#         'random_state': 48,
#         'n_estimators': 20000,
#         'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
#         'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
#         'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
#         'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
#         'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
#         'max_depth': trial.suggest_categorical('max_depth', [10,20,100]),
#         'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
#         'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
#         'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100)
# }

# rf_params =   {
#             'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
#             'max_depth': trial.suggest_int('max_depth', 4, 50),
#             'min_samples_split': trial.suggest_int('min_samples_split', 1, 150),
#             'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 60),
#         }

In [19]:
#Define models

models = {
            "xgb": xgb.XGBRegressor(),
            #"xgb_exact": xgb.XGBRegressor(**xgb_exact_params),
            #"xgb_approx": xgb.XGBRegressor(**xgb_approx_params),
            "lgb": lgb.LGBMRegressor(),
            #"lgb2": lgb.LGBMRegressor(**lgb2_params),
            #"lgb3": lgb.LGBMRegressor(**lgb3_params),
            "cat": CatBoostRegressor(),
            #"cat2": CatBoostRegressor(**cb2_params),
           # "cat3": CatBoostRegressor(**cb3_params),
            #"cat_sym": CatBoostRegressor(**cb_sym_params),
            #"cat_loss": CatBoostRegressor(**cb_loss_params),
            "Ridge": RidgeCV(),
            "Lasso": LassoCV(),
            "RandomForestRegressor": RandomForestRegressor(n_estimators=200, random_state=self.random_state, n_jobs=-1),
            "PLSRegression": PLSRegression(n_components=10, max_iter=2000),
            #"PassiveAggressiveRegressor": PassiveAggressiveRegressor(max_iter=3000, tol=1e-3, n_iter_no_change=30, random_state=self.random_state),
            "GradientBoostingRegressor": GradientBoostingRegressor(n_estimators=2000, learning_rate=0.05, loss="absolute_error", random_state=self.random_state),
           "HistGradientBoostingRegressor": HistGradientBoostingRegressor(max_iter=self.n_estimators, learning_rate=0.01, loss="absolute_error", n_iter_no_change=300,random_state=self.random_state),
            #"ARDRegression": ARDRegression(n_iter=1000),
           # "HuberRegressor": HuberRegressor(max_iter=3000),
            #"KNeighborsRegressor": KNeighborsRegressor(n_neighbors=5, n_jobs=-1)
        }

In [1]:
#XGB Optuna
def objective(trial):
    
    xgb_params = {
        #'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 10000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'random_state': trial.suggest_categorical('random_state', [2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    
    global train_X
    global train_y
    
    train_x, valid_x, y_train, y_valid = train_test_split(train_X, train_y, test_size=0.25)
    
    model = xgb.XGBRegressor(**xgb_params)
    pipe = make_pipeline(preprocessor, model)
    model.fit(train_x, y_train)
    val_pred = model.predict(valid_x)
    val_metric = mean_absolute_error( y_valid, val_pred)
    return val_metric

In [23]:
study = optuna.create_study( direction="minimize")
study.optimize(objective, n_trials=20, timeout=600 )

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-05-15 05:28:28,978] A new study created in memory with name: no-name-94e4c4a8-665c-49df-b493-cdf63ff0c9ad
/tmp/ipykernel_35/1249193205.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
/tmp/ipykernel_35/1249193205.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
[I 2023-05-15 05:29:50,374] Trial 0 finished with value: 374.5050242194268 and parameters: {'lambda': 0.08319533243642291, 'alpha': 6.384690563583655, 'colsample_bytree': 0.4, 'subsample': 1.0, 'learning_rate': 0.02, 'max_depth': 13, 'random_state': 2020, 'min_c

Number of finished trials: 8
Best trial:
  Value: 354.09378324242675
  Params: 
    lambda: 0.002534095883438017
    alpha: 0.21009936724141606
    colsample_bytree: 1.0
    subsample: 0.5
    learning_rate: 0.01
    max_depth: 5
    random_state: 2020
    min_child_weight: 231


In [44]:
#LGBM Optuna
def objective(trial):
    
    lgb_params = {
        'metric': 'mae', 
        'random_state': 48,
        'n_estimators': 10000,
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        'max_depth': trial.suggest_categorical('max_depth', [10,20,100]),
        'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100)
}
    global train_X
    global train_y
    
    train_x, valid_x, y_train, y_valid = train_test_split(train_X, train_y, test_size=0.25)
    
#     lg_train = lgb.Dataset(train_x, label = y_train)
#     lg_validation = lgb.Dataset(valid_x, label= y_valid)
#     evals_result_lgbm = {}
    
    model = lgb.LGBMRegressor(**lgb_params)
    pipe = make_pipeline(preprocessor, model)
#     model.fit(lgb_params, lg_train, 5000,
#                       valid_sets=[lg_train, lg_validation],
#                       early_stopping_rounds=100,
#                       verbose_eval=150,
#                       evals_result=evals_result_lgbm )
    model.fit(train_x, y_train)
    val_pred = model.predict(valid_x)
    val_metric = mean_absolute_error( y_valid, val_pred)
    return val_metric

In [ ]:
study = optuna.create_study( direction="minimize")
study.optimize(objective, n_trials=25, timeout=600 )

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-05-15 06:48:24,696] A new study created in memory with name: no-name-785f2f66-a41b-4c04-ba1f-cd6aab2968ae
/tmp/ipykernel_35/877680049.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
/tmp/ipykernel_35/877680049.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
[I 2023-05-15 06:49:17,845] Trial 0 finished with value: 352.27471937458915 and parameters: {'reg_alpha': 0.6040853586578172, 'reg_lambda': 1.2665389203832345, 'colsample_bytree': 1.0, 'subsample': 0.7, 'learning_rate': 0.006, 'max_depth': 10, 'num

In [34]:
#Catboost Optuna

def objective(trial):
    
    cat_params  = {
        'loss_function': 'RMSE',
        #'task_type': 'GPU',
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
        'max_bin': trial.suggest_int('max_bin', 200, 400),
        #'rsm': trial.suggest_uniform('rsm', 0.3, 1.0),
        'subsample': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.006, 0.018),
        'n_estimators':  25000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15]),
        'random_state': trial.suggest_categorical('random_state', [2020]),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 300),
        'verbose': 1000
    }
    
    global train_X
    global train_y
    
    train_x, valid_x, y_train, y_valid = train_test_split(train_X, train_y, test_size=0.25)
    
    model = CatBoostRegressor(**cat_params)
    pipe = make_pipeline(preprocessor, model)
    model.fit(train_x, y_train, verbose = 200)
    val_pred = model.predict(valid_x)
    val_metric = mean_absolute_error( y_valid, val_pred)
    return val_metric

In [35]:
study = optuna.create_study( direction="minimize")
study.optimize(objective, n_trials=10, timeout=600 )

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-05-15 06:35:48,804] A new study created in memory with name: no-name-add13e19-0716-4852-8bbd-eda9e3320091
/tmp/ipykernel_35/209665615.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
/tmp/ipykernel_35/209665615.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'subsample': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
/tmp/ipykernel_35/209665615.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` inste

0:	learn: 1320.5263425	total: 5.43ms	remaining: 2m 15s
200:	learn: 563.2386719	total: 1.42s	remaining: 2m 54s
400:	learn: 526.1076801	total: 2.77s	remaining: 2m 50s
600:	learn: 508.3052573	total: 4.1s	remaining: 2m 46s
800:	learn: 491.9076617	total: 5.42s	remaining: 2m 43s
1000:	learn: 475.7025413	total: 6.7s	remaining: 2m 40s
1200:	learn: 459.3338602	total: 8.1s	remaining: 2m 40s
1400:	learn: 444.2434572	total: 9.46s	remaining: 2m 39s
1600:	learn: 430.4593302	total: 10.8s	remaining: 2m 38s
1800:	learn: 418.2878387	total: 12.2s	remaining: 2m 37s
2000:	learn: 407.3378463	total: 13.6s	remaining: 2m 36s
2200:	learn: 396.8156235	total: 15.1s	remaining: 2m 36s
2400:	learn: 386.8408760	total: 16.5s	remaining: 2m 35s
2600:	learn: 377.6763989	total: 17.8s	remaining: 2m 33s
2800:	learn: 368.2141844	total: 19.3s	remaining: 2m 33s
3000:	learn: 359.4844687	total: 20.6s	remaining: 2m 31s
3200:	learn: 351.1929268	total: 22s	remaining: 2m 29s
3400:	learn: 343.4210020	total: 23.3s	remaining: 2m 27s
36

[I 2023-05-15 06:38:39,322] Trial 0 finished with value: 372.6660118522965 and parameters: {'l2_leaf_reg': 0.29137596334184773, 'max_bin': 217, 'bagging_fraction': 0.791625271863861, 'learning_rate': 0.011853683515091477, 'max_depth': 9, 'random_state': 2020, 'min_data_in_leaf': 46}. Best is trial 0 with value: 372.6660118522965.
/tmp/ipykernel_35/209665615.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
/tmp/ipykernel_35/209665615.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'subsample': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
/tmp/ipykernel_35/209665615.

0:	learn: 1319.1639675	total: 19.2ms	remaining: 8m
200:	learn: 512.7826178	total: 3.48s	remaining: 7m 10s
400:	learn: 464.3098112	total: 7.4s	remaining: 7m 33s
600:	learn: 425.7945121	total: 11s	remaining: 7m 27s
800:	learn: 390.5205555	total: 15.1s	remaining: 7m 36s
1000:	learn: 360.0243382	total: 19.3s	remaining: 7m 42s
1200:	learn: 334.4081945	total: 23.5s	remaining: 7m 45s
1400:	learn: 310.0476257	total: 27.7s	remaining: 7m 47s
1600:	learn: 287.1862352	total: 31.8s	remaining: 7m 44s
1800:	learn: 267.0504965	total: 35.7s	remaining: 7m 39s
2000:	learn: 249.7831462	total: 39.8s	remaining: 7m 37s
2200:	learn: 233.7958011	total: 44.2s	remaining: 7m 37s
2400:	learn: 219.1127755	total: 48.4s	remaining: 7m 35s
2600:	learn: 206.0232265	total: 52.8s	remaining: 7m 34s
2800:	learn: 194.7555724	total: 57s	remaining: 7m 31s
3000:	learn: 184.4985711	total: 1m 1s	remaining: 7m 29s
3200:	learn: 175.4061150	total: 1m 5s	remaining: 7m 25s
3400:	learn: 166.8642701	total: 1m 9s	remaining: 7m 20s
3600:	

[W 2023-05-15 06:41:40,823] Trial 1 failed with parameters: {'l2_leaf_reg': 0.007602108511074881, 'max_bin': 293, 'bagging_fraction': 0.8311859545565963, 'learning_rate': 0.01760897569543684, 'max_depth': 11, 'random_state': 2020, 'min_data_in_leaf': 24} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_35/209665615.py", line 27, in objective
    model.fit(train_x, y_train, verbose = 200)
  File "/opt/conda/lib/python3.10/site-packages/catboost/core.py", line 5730, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline,
  File "/opt/conda/lib/python3.10/site-packages/catboost/core.py", line 2355, in _fit
    self._train(
  File "/opt/conda/lib/python3.10/site-packages/catboost/core.py", line 1759, in _train
    

KeyboardInterrupt: 